<a href="https://colab.research.google.com/github/MuhammadIrzam447/NewEncodings/blob/main/Valid_29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
import requests
import torch
from PIL import Image
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms
import torchvision.models as models
from tqdm import tqdm

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


# Fused Validation

In [ ]:
import os

image_file_paths = []
genre_labels = []

image_folder_add = "/content/fused/test"
labels_file = "/content/fused_test_label.txt"

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces

        if not (filename.endswith("_1.png") or filename.endswith("_2.png")):
            image_path = os.path.join(image_folder_add, filename)
            image_file_paths.append(image_path)
            genre_labels.append(labels)


In [ ]:
len(image_file_paths)

In [ ]:
 image_file_paths[0:10]

# 100% Missing Modality

In [ ]:
# _3.png is for Encoded Image
# _4.png is for Actual Image

In [ ]:
import os

image_file_paths = []
genre_labels = []

image_folder_add = "/content/joint/test"
labels_file = "/content/joint_test_label.txt"

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces

        if filename.endswith("_3.png"):
            image_path = os.path.join(image_folder_add, filename)
            image_file_paths.append(image_path)
            genre_labels.append(labels)

In [ ]:
len(image_file_paths)

In [ ]:
image_file_paths[0:10]

# Partial Missing Modality

Load Fused

In [ ]:
import os

fused_image_file_paths = []
fused_genre_labels = []

image_folder_add = "/content/fused/test"
labels_file = "/content/fused_test_label.txt"

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces

        if not (filename.endswith("_1.png") or filename.endswith("_2.png")):
            image_path = os.path.join(image_folder_add, filename)
            fused_image_file_paths.append(image_path)
            fused_genre_labels.append(labels)

In [ ]:
len(fused_image_file_paths)

Load Image Only

In [ ]:
# _3.png is for Encoded Image
# _4.png is for Actual Image

In [ ]:
import os

img_file_paths = []
img_genre_labels = []

image_folder_add = "/content/joint/test"
labels_file = "/content/joint_test_label.txt"

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces

        if filename.endswith("_4.png"):
            image_path = os.path.join(image_folder_add, filename)
            img_file_paths.append(image_path)
            img_genre_labels.append(labels)

In [ ]:
len(img_file_paths)

Random Selection

In [ ]:
percentage = 0.5 # Fused Availability

In [ ]:
import random

total_samples = len(fused_image_file_paths)
num_samples_from_fused = int(percentage * total_samples)
num_samples_from_image = total_samples - num_samples_from_fused

In [ ]:
print("total_samples", total_samples)
print("num_samples_from_fused", num_samples_from_fused)
print("num_samples_from_image", num_samples_from_image)

In [ ]:
random_indices_fused = random.sample(range(total_samples), num_samples_from_fused)
remaining_indices_image = [i for i in range(total_samples) if i not in random_indices_fused]


image_file_paths = [fused_image_file_paths[i] for i in random_indices_fused] + [img_file_paths[i] for i in remaining_indices_image]
genre_labels = [fused_genre_labels[i] for i in random_indices_fused] + [img_genre_labels[i] for i in remaining_indices_image]

In [ ]:
image_file_paths[7000]

In [ ]:
genre_labels[7000]

# EDA

In [ ]:
from collections import defaultdict

label_counts = defaultdict(int)

for labels in genre_labels:
    for label in labels:
        label_counts[label] += 1

In [ ]:
label_count_list = [(label, count) for label, count in label_counts.items()]
sorted_label_count_list = sorted(label_count_list, key=lambda x: x[1], reverse=True)

for label, count in sorted_label_count_list:
    print(f"{label}: {count}")

print("Total Labels: ", len(label_count_list))

In [ ]:
min_label_count = 20
valid_labels = [label for label, count in label_counts.items() if count >= min_label_count]
valid_labels = sorted(list(valid_labels))

In [ ]:
valid_labels, len(valid_labels)

MultiHot Encoding

In [ ]:
multi_hot_labels = []

for labels in genre_labels:
    multi_hot = [1 if label in labels else 0 for label in valid_labels]
    multi_hot_labels.append(multi_hot)

In [ ]:
multi_hot_labels[1]

In [ ]:
genre_labels[1]

# Create Dataset and Validation (ResNet Fused)

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class CustomMultiLabelDataset(Dataset):
    def __init__(self, image_file_paths, multi_encoded_labels, transform=None):
        self.image_file_paths = image_file_paths
        self.multi_encoded_labels = multi_encoded_labels
        self.transform = transform

    def __len__(self):
        return len(self.image_file_paths)

    def __getitem__(self, idx):
        image_path = self.image_file_paths[idx]
        image = Image.open(image_path)

        labels = self.multi_encoded_labels[idx]
        labels = torch.tensor(labels, dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, labels

# Define data transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Create a custom dataset
val_dataset = CustomMultiLabelDataset(image_file_paths, multi_hot_labels, transform=transform)
print(len(val_dataset))

Load Model and Validation Loop (ResNet)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import os
import torch
import torch.nn.functional as F

save_dir = '/content/'
load_path = os.path.join(save_dir, '10_model.pth')

model = models.resnet101(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, len(valid_labels))
model.load_state_dict(torch.load(load_path))

model.eval()
model.to(device)

In [ ]:
batch_size = 32
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
model.eval()
img_predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)

        # img_predictions.extend((outputs > 0.5).int().cpu().numpy())
        img_predictions.extend(F.sigmoid(outputs).cpu().numpy())
        true_labels.extend(labels.int().cpu().numpy())

In [ ]:
import numpy as np
img_predictions = np.array(img_predictions)

In [ ]:
threshold = 0.5
predictions = (img_predictions > threshold).astype(int)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='macro')
recall = recall_score(true_labels, predictions, average='macro')
f1 = f1_score(true_labels, predictions, average='macro')

In [ ]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

In [ ]:
print(classification_report(true_labels, predictions))

# Create Dataset and Validation (ViT Fused)

In [ ]:
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm
import torch
import torch.nn.functional as F

In [ ]:
val_data = {'image': image_file_paths, 'label': multi_hot_labels}
ds_val = Dataset.from_dict(val_data)

In [ ]:
ds_val

In [ ]:
ds_val['image'][0]

In [ ]:
labels = valid_labels
labels

Load Model

In [ ]:
model = ViTForImageClassification.from_pretrained(f"/content/Model/Models-Train-18/checkpoint-81000").to(device)
image_processor = ViTImageProcessor.from_pretrained(f"/content/Model/Models-Train-18/checkpoint-81000")

In [ ]:
import PIL.Image as pil

def transform(examples):
  inputs = image_processor([pil.open(img).convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]
  return inputs

In [ ]:
val_dataset = ds_val.with_transform(transform)

In [ ]:
val_dataset

In [ ]:
sample = val_dataset[0]

In [ ]:
sample

In [ ]:
for item in val_dataset:
  print(item['pixel_values'].shape)
  print(item["labels"])
  break

In [ ]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [ ]:
batch_size = 32
valid_dataset_loader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [ ]:
model.eval()

vit_predictions, labels = [], []

for batch in valid_dataset_loader:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device).float()

    outputs = model(pixel_values=pixel_values, labels=label_ids)

    logits = outputs.logits.detach().cpu()

    # predictions.extend((logits > 0.5).int().cpu().numpy())
    vit_predictions.extend(F.sigmoid(logits).cpu().numpy())
    labels.extend(label_ids.int().cpu().numpy())

In [ ]:
import numpy as np
vit_predictions = np.array(vit_predictions)

In [ ]:
threshold = 0.5
predictions = (vit_predictions > threshold).astype(int)

In [ ]:
accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='macro')
recall = recall_score(labels, predictions, average='macro')
f1 = f1_score(labels, predictions, average='macro')

In [ ]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

In [ ]:
print(classification_report(labels, predictions))

# PreProcessing for Joint Missing Modality

In [ ]:
# _3.png is for Encoded Image
# _4.png is for Actual Image

In [3]:
import os

image_file_paths = []
genre_labels = []
encoded_file_paths = []
encoded_genre_labels = []

image_folder_add = "/content/MMLearning/data/imdb/joint384/multimodal/test"
labels_file = "/content/MMLearning/data/imdb/joint384/joint_test_label.txt"

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces
        image_path = os.path.join(image_folder_add, filename)

        if filename.endswith("_4.png"):
            image_file_paths.append(image_path)
            genre_labels.append(labels)

        if filename.endswith("_3.png"):
            encoded_file_paths.append(image_path)
            encoded_genre_labels.append(labels)

In [4]:
len(image_file_paths)

7799

In [5]:
len(encoded_file_paths)

7799

In [6]:
image_file_paths[0:5]

['/content/MMLearning/data/imdb/joint384/multimodal/test/0078718_4.png',
 '/content/MMLearning/data/imdb/joint384/multimodal/test/0089003_4.png',
 '/content/MMLearning/data/imdb/joint384/multimodal/test/0098136_4.png',
 '/content/MMLearning/data/imdb/joint384/multimodal/test/0057693_4.png',
 '/content/MMLearning/data/imdb/joint384/multimodal/test/0385330_4.png']

In [7]:
encoded_file_paths[0:5]

['/content/MMLearning/data/imdb/joint384/multimodal/test/0078718_3.png',
 '/content/MMLearning/data/imdb/joint384/multimodal/test/0089003_3.png',
 '/content/MMLearning/data/imdb/joint384/multimodal/test/0098136_3.png',
 '/content/MMLearning/data/imdb/joint384/multimodal/test/0057693_3.png',
 '/content/MMLearning/data/imdb/joint384/multimodal/test/0385330_3.png']

In [8]:
from collections import defaultdict

label_counts = defaultdict(int)

for labels in genre_labels:
    for label in labels:
        label_counts[label] += 1

In [9]:
label_count_list = [(label, count) for label, count in label_counts.items()]
sorted_label_count_list = sorted(label_count_list, key=lambda x: x[1], reverse=True)

for label, count in sorted_label_count_list:
    print(f"{label}: {count}")

print("Total Labels: ", len(label_count_list))

Drama: 4142
Comedy: 2611
Romance: 1590
Thriller: 1567
Crime: 1163
Action: 1044
Horror: 825
Adventure: 821
Documentary: 629
Mystery: 617
Sci-Fi: 586
Fantasy: 585
Family: 518
Biography: 411
War: 401
History: 345
Music: 311
Animation: 306
Musical: 253
Western: 210
Sport: 191
Short: 142
Film-Noir: 102
News: 19
Adult: 1
Total Labels:  25


In [10]:
min_label_count = 20
valid_labels = [label for label, count in label_counts.items() if count >= min_label_count]
valid_labels = sorted(list(valid_labels))

In [11]:
valid_labels, len(valid_labels)

(['Action',
  'Adventure',
  'Animation',
  'Biography',
  'Comedy',
  'Crime',
  'Documentary',
  'Drama',
  'Family',
  'Fantasy',
  'Film-Noir',
  'History',
  'Horror',
  'Music',
  'Musical',
  'Mystery',
  'Romance',
  'Sci-Fi',
  'Short',
  'Sport',
  'Thriller',
  'War',
  'Western'],
 23)

In [12]:
num_of_classes = len(valid_labels)

Multi Hot Encoding

In [13]:
img_multi_hot_labels = []

for labels in genre_labels:
    multi_hot = [1 if label in labels else 0 for label in valid_labels]
    img_multi_hot_labels.append(multi_hot)

In [14]:
enc_multi_hot_labels = []

for labels in encoded_genre_labels:
    multi_hot = [1 if label in labels else 0 for label in valid_labels]
    enc_multi_hot_labels.append(multi_hot)

Sample Check

In [ ]:
index = 10

In [ ]:
image_file_paths[index], genre_labels[index] , img_multi_hot_labels[index]

In [ ]:
encoded_file_paths[index], encoded_genre_labels[index] , enc_multi_hot_labels[index]

# Partial Missing Modality with ResNet (Joint)

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch
import torch.nn.functional as F

class CustomMultiLabelDataset(Dataset):
    def __init__(self, image_file_paths, multi_encoded_labels, transform=None):
        self.image_file_paths = image_file_paths
        self.multi_encoded_labels = multi_encoded_labels
        self.transform = transform

    def __len__(self):
        return len(self.image_file_paths)

    def __getitem__(self, idx):
        image_path = self.image_file_paths[idx]
        image = Image.open(image_path)

        labels = self.multi_encoded_labels[idx]
        labels = torch.tensor(labels, dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, labels

# Define data transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Create a custom dataset
img_dataset = CustomMultiLabelDataset(image_file_paths, img_multi_hot_labels, transform=transform)
enc_dataset = CustomMultiLabelDataset(encoded_file_paths, enc_multi_hot_labels, transform=transform)

In [ ]:
len(img_dataset) , len(enc_dataset)

In [ ]:
batch_size = 32
img_loader = DataLoader(img_dataset, batch_size=batch_size, shuffle=False)
enc_loader = DataLoader(enc_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
num_batches = len(enc_loader)
num_batches_with_logits_3 = int(num_batches * 0.3)  # ___% of batches that you want to include (30 for now)
print("Total Batches: ", num_batches)
print("Missing Modaility Batches: ", num_batches_with_logits_3)

In [ ]:
import random

controller = random.sample(range(1, num_batches+1), num_batches_with_logits_3)
controller = sorted(controller)

In [ ]:
len(controller)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import os

save_dir = '/content/Model/Models-Train-22/'
load_path = os.path.join(save_dir, '29_model.pth')

model = models.resnet101(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, len(valid_labels))
model.load_state_dict(torch.load(load_path))

model.eval()
model.to(device)

In [ ]:
model.eval()
img_predictions = []
img_true_labels = []

with torch.no_grad():
    for inputs, labels in img_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)

        img_predictions.extend(F.sigmoid(outputs).cpu().numpy())
        img_true_labels.extend(labels.int().cpu().numpy())

In [ ]:
enc_predictions = []
enc_true_labels = []

with torch.no_grad():
    batch_idx = 0
    for inputs, labels in enc_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        if batch_idx in controller:
            outputs = model(inputs)
        else:
            length = len(labels)
            outputs = torch.zeros((length, num_of_classes), device="cpu")

        enc_predictions.extend(F.sigmoid(outputs).cpu().numpy())
        enc_true_labels.extend(labels.int().cpu().numpy())
        batch_idx = batch_idx + 1

In [ ]:
import numpy as np
predicted_avg = (np.array(img_predictions) + np.array(enc_predictions)) / 2.0

In [ ]:
# Apply the threshold of 0.5
threshold = 0.5
predictions = (predicted_avg > threshold).astype(int)

In [ ]:
true_labels = img_true_labels

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='macro')
recall = recall_score(true_labels, predictions, average='macro')
f1 = f1_score(true_labels, predictions, average='macro')

In [ ]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

In [ ]:
print(classification_report(true_labels, predictions))

# Partial Missing Modality with ViT (Joint)

In [17]:
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm
import torch
import torch.nn.functional as F

In [18]:
from datasets import Dataset
from torch.utils.data import DataLoader

img_data = {'image': image_file_paths, 'label': img_multi_hot_labels}
ds_img = Dataset.from_dict(img_data)

enc_data = {'image': encoded_file_paths, 'label': enc_multi_hot_labels}
ds_enc = Dataset.from_dict(enc_data)

In [19]:
import PIL.Image as pil

def transform(examples):
  inputs = image_processor([pil.open(img).convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]
  return inputs

In [20]:
img_dataset = ds_img.with_transform(transform)
enc_dataset = ds_enc.with_transform(transform)

In [21]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [22]:
batch_size = 8
img_loader = DataLoader(img_dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)
enc_loader = DataLoader(enc_dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [23]:
num_batches = len(enc_loader)
num_batches_with_logits_3 = int(num_batches * 1)  # ___% of batches that you want to include (30 for now)
print("Total Batches: ", num_batches)
print("Missing Modaility Batches: ", num_batches_with_logits_3)

Total Batches:  975
Missing Modaility Batches:  975


In [24]:
import random

controller = random.sample(range(1, num_batches+1), num_batches_with_logits_3)
controller = sorted(controller)

In [25]:
len(controller)

975

Load Model

In [26]:
model = ViTForImageClassification.from_pretrained(f"/content/MMLearning/data/Models/Model-17/checkpoint-99999").to(device)
image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-384")

loading configuration file /content/MMLearning/data/Models/Model-17/checkpoint-99999/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-384",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "Action",
    "1": "Adventure",
    "2": "Animation",
    "3": "Biography",
    "4": "Comedy",
    "5": "Crime",
    "6": "Documentary",
    "7": "Drama",
    "8": "Family",
    "9": "Fantasy",
    "10": "Film-Noir",
    "11": "History",
    "12": "Horror",
    "13": "Music",
    "14": "Musical",
    "15": "Mystery",
    "16": "Romance",
    "17": "Sci-Fi",
    "18": "Short",
    "19": "Sport",
    "20": "Thriller",
    "21": "War",
    "22": "Western"
  },
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Action": 0,
    "Adventure": 1,
    "Animati

In [27]:
model.eval()
img_predictions = []
img_true_labels = []

for batch in img_loader:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device).float()

    outputs = model(pixel_values=pixel_values, labels=label_ids)

    logits = outputs.logits.detach().cpu()
    # probabilities = torch.softmax(logits, dim=1)

    img_predictions.extend(F.sigmoid(logits).cpu().numpy())
    img_true_labels.extend(label_ids.int().cpu().numpy())

In [28]:
enc_predictions = []
enc_true_labels = []

batch_idx = 0
for batch in enc_loader:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device).float()

    if batch_idx in controller:
        outputs = model(pixel_values=pixel_values, labels=label_ids)
        logits = outputs.logits.detach().cpu()
    else:
        length = len(label_ids)
        logits = torch.zeros((length, num_of_classes), device="cpu")

    # probabilities = torch.softmax(logits, dim=1)
    enc_predictions.extend(F.sigmoid(logits).cpu().numpy())
    enc_true_labels.extend(label_ids.int().cpu().numpy())
    batch_idx = batch_idx + 1

In [29]:
import numpy as np
predicted_avg = (np.array(img_predictions) + np.array(enc_predictions)) / 2.0

In [30]:
# Apply the threshold of 0.5
threshold = 0.5
predictions = (predicted_avg >= threshold).astype(int)

In [31]:
true_labels = img_true_labels

In [32]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='macro')
recall = recall_score(true_labels, predictions, average='macro')
f1 = f1_score(true_labels, predictions, average='macro')

In [33]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

Accuracy: 0.12347736889344788
Precision: 0.5751509347749889
Recall: 0.2112859875163003
F1-Score: 0.2789942829804703


In [ ]:
print(classification_report(true_labels, predictions))

In [ ]:
valid_labels

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
import numpy as np

true_labels = np.array(true_labels, dtype=int)
predictions = np.array(predictions, dtype=int)
print(multilabel_confusion_matrix(true_labels, predictions, labels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]))
# print(multilabel_confusion_matrix(true_labels, predictions))